In [1]:
!pip install pyspark 
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 21.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=5ca9b8a8574e5b4c8f4db4b1a88be394146341e8ecf58b92cad28a8416454d4c
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
cur_path = "/content/drive/MyDrive/bds/"
os.chdir(cur_path)
!pwd

/content/drive/MyDrive/bds


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("netflix").getOrCreate()

In [6]:
df = spark.read.csv(cur_path + 'netflix_titles.txt',inferSchema=True,header=True,sep='\t')

In [7]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- show_id: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [8]:
df.select('cast').show()

+--------------------+
|                cast|
+--------------------+
|Alan Marriott, An...|
|    Jandino Asporaat|
|Peter Cullen, Sum...|
|Will Friedle, Dar...|
|Nesta Cooper, Kat...|
|Alberto Ammann, E...|
|Antonio Banderas,...|
|     Fabrizio Copano|
|                null|
|James Franco, Kat...|
|       Joaquín Reyes|
|Jim Sturgess, Sam...|
|Damandeep Singh B...|
|Damandeep Singh B...|
|Damandeep Singh B...|
|Damandeep Singh B...|
|Rishi Gambhir, Sm...|
|Damandeep Singh B...|
|Damandeep Singh B...|
|Karl Glusman, Kla...|
+--------------------+
only showing top 20 rows



In [9]:
import itertools
from pyspark import SparkContext

In [10]:
df = df.na.drop(subset=['cast'])

In [11]:
cast = df.select("cast").rdd.flatMap(lambda x:x)

In [12]:
cast_pair = cast.map(lambda x:x.split(", ")).map(lambda x:list(itertools.combinations(x,2))).collect()

In [13]:
from pyspark.sql.types import *

In [14]:
spark = SparkSession.builder.appName('df2').getOrCreate()

In [15]:
field = [
    StructField("cast1", StringType(), True),
    StructField("cast2", StringType(), True),
]
schema = StructType(field)
df2 = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema)
df2.printSchema()

root
 |-- cast1: string (nullable = true)
 |-- cast2: string (nullable = true)



In [16]:
df2.show()

+-----+-----+
|cast1|cast2|
+-----+-----+
+-----+-----+



In [ ]:
for i in cast_pair:
  if i:
    for element in i:
      #print(type(element))
      newRow = spark.createDataFrame([sorted(element)])
      df2 = df2.union(newRow)

In [17]:
file = open("cast.txt","a+")
for i in cast_pair:
  if i:
    for element in i:
      element_list = sorted(element)
      pair = element_list[0]+','+ element_list[1]+'\n'
      file.write(pair)
file.close()